# **Milestone 1: Single Character Classifier**

This is the notebook of our prototype single character classifier. This is the first milestone in our project, because being able to identify single characters is a prerequisite to identifying the characters used in the captcha 

In [ ]:
#modules
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

use_cuda = True

# **Data Loading**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def load_data(batch_size = 64):
    
   
  
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_set = torchvision.datasets.ImageFolder(root='/content/gdrive/MyDrive/path/to/train', transform=transform)
    #print(train_set)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_set = torchvision.datasets.ImageFolder(root='/content/gdrive/MyDrive/path/to/val', transform=transform)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

    return train_loader, val_loader

# **The CNN**

In [ ]:
class char_cnn(nn.Module):
    def __init__(self):
        super(char_cnn, self).__init__()
        self.name = "c_cnn"
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 5)
        self.fc1 = nn.Linear(13*13*10, 100) # initial dimensions are a 64x64 pixel image
        self.fc2 = nn.Linear(100, 62)       # there are 36 possible characters, 52 letters (A-Z, and a-z) and 10 digits (0-9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 13*13*10)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# **Training**

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [ ]:
# Training Curve
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """
    import matplotlib.pyplot as plt
    train_acc = np.loadtxt("{}_train_acc.csv".format(path))
    train_err = np.loadtxt("{}_train_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    val_acc = np.loadtxt("{}_val_acc.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    n = len(train_err) # number of epochs
    plt.plot(range(1,n+1), train_err, label="Train")
    plt.plot(range(1,n+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Acc")
    plt.plot(range(1,n+1), train_acc, label="Train")
    plt.plot(range(1,n+1), val_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Acc")
    plt.plot(range(1,n+1), train_acc, label="Train")
    plt.plot(range(1,n+1), val_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

In [ ]:
def evaluate(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0
    corr = 0.0
    #net.cuda()
    for i, data in enumerate(loader, 0):
        imgs, labels = data


        imgs = imgs.cuda()
        labels = labels.cuda()


        out = net(imgs)
        loss = criterion(out, labels)

        pred = out.max(1, keepdim=True)[1]
        corr = pred.eq(labels.view_as(pred)).sum().item()
        total_err += len(labels)-int(corr)
        total_loss += loss.item()
        total_epoch += len(labels)
    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)
    return err, loss

In [ ]:
def train(model,batch_size=64, learning_rate=0.01,num_epochs=50):
    torch.manual_seed(1000)
    train_loader, val_loader = load_data(batch_size)
  
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)


    # Set up some numpy arrays to store the training/test loss/erruracy
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    train_acc = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    # training
    n = 0 # the number of iterations
    print(("use_cuda is {}, torch.cuda.is_available() is {}").format(use_cuda, torch.cuda.is_available()))
    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_train_err = 0.0
        total_train_acc = 0.0
        total_epoch = 0
        corr = 0.0
        #print("epoch start")
        #for i in range(4):
        #for data in iter(train_loader):
        for i, data in enumerate(train_loader, 0):
            #print("start")
            imgs, labels = data
            #print(imgs)
            #print(labels)
            #############################################
            #To Enable GPU Usage
            if use_cuda and torch.cuda.is_available():
                imgs = imgs.cuda()
                labels = labels.cuda()
            #############################################
            #print("training start")
            out = model(imgs)             # forward pass
            #print(out)
            optimizer.zero_grad()         # a clean up step for PyTorch

            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            #print("training end")
            
            pred = out.max(1, keepdim=True)[1]
            corr = pred.eq(labels.view_as(pred)).sum().item()
            #print(("corr is {}, int(corr) is {}").format(corr, int(corr)))

            total_train_err += len(labels)-int(corr)
            total_train_acc += (int(corr))


            total_train_loss += loss.item()
            total_epoch += len(labels)
            n = n+1


            #print(("n is {}, i is {}").format(n,i))
            #print("loss, accuracy, err calc done")
        #print("epoch end")
        train_err[epoch] = float(total_train_err) / total_epoch
        train_acc[epoch] = float(total_train_acc) / total_epoch
        train_loss[epoch] = float(total_train_loss) / (i+1)
        #train_loss[epoch] = float(total_train_loss)
        #model.cuda()
        val_err[epoch], val_loss[epoch] = evaluate(model, val_loader, criterion)
        val_acc[epoch] = 1-val_err[epoch]
        print(("Epoch {}: Train err: {}, Train loss: {}, Train acc: {} |"+
               "Validation err: {}, Validation loss: {}, Validation Acc: {}").format(
                   epoch + 1,
                   train_err[epoch],
                   train_loss[epoch],
                   train_acc[epoch],
                   val_err[epoch],
                   val_loss[epoch],
                   val_acc[epoch]))
        # Save the current model (checkpoint) to a file
        model_path = get_model_name(model.name, batch_size, learning_rate, epoch)
        torch.save(model.state_dict(), model_path)
    print('Finished Training')
    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)     
    np.savetxt("{}_train_acc.csv".format(model_path), train_acc) 
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)
    np.savetxt("{}_val_acc.csv".format(model_path), val_acc)

Now we actually instantiate / train:

In [ ]:
sccnn = char_cnn()
sccnn.cuda()
train(sccnn)
model_path = get_model_name("c_cnn", batch_size=64, learning_rate=0.01, epoch=49)
plot_training_curve(model_path)